In [121]:
# Execute in Windows Prompt or Terminal (MAC)
# !gcloud auth application-default login

In [ ]:
!gcloud config set project hca-hin-dev-cur-parallon

Updated property [core/project].


In [123]:
from google.cloud import bigquery
import google.auth
import pandas as pd
from datetime import datetime

In [124]:
import os
os.environ["HTTP_PROXY"] = "proxy.nas.medcity.net:80"
os.environ["HTTPS_PROXY"] = "proxy.nas.medcity.net:80"

In [125]:
credentials, project_id = google.auth.default(scopes=["https://www.googleapis.com/auth/cloud-platform"])
print(project_id)
bq_client = bigquery.Client(credentials=credentials, project=project_id)

C:\Users\KHU9683\AppData\Roaming\Python\Python311\site-packages\google\auth\_default.py:83: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


hca-hin-prod-cur-parallon


In [126]:
# dataset_name = 'edwclm_staging'
# table_name = 'fact_code'
# dataset = bq_client.dataset(dataset_name)
# table_ref = dataset.table(table_name)
# table = bq_client.get_table(table_ref)
# print(table)

In [127]:
input_path = "OutputFiles\\secondary_billing_patients.csv"
output_path = "OutputFiles\\secondary_billing_patients_bqlookup.csv"

In [128]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = '\n'.join(file_data)
        file.write(file_string)

In [129]:
 
def lookup_bq_data():
    dt1 = datetime.now()
    source_df = pd.read_csv(input_path, index_col=None, 
    converters={'PAS_COID': str,'Unit_Num': str,'IPlan': str,'IPlan2': str})
    try:
        source_df.sort_values(by=['Unit_Num','Patient_Acct_Num'], inplace=True)
        source_df.reset_index(drop=True, inplace=True)
        source_df = source_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

        for unit_num in source_df['Unit_Num'].unique():
            patient_acct_num_list = (',').join(list(map(str,source_df.loc[source_df['Unit_Num'] == unit_num,'Patient_Acct_Num'].values)))
            target_query = "SELECT pat_acct_num, iplan_id_ins1, iplan_id_ins2, patient_dw_id FROM edwpbs_base_views.fact_patient WHERE unit_num = '{}' AND pat_acct_num IN ({})".format(unit_num, patient_acct_num_list)
            target_rs = bq_client.query(target_query, project=project_id, location='US').result()
            for target_row in target_rs:
                patient_acct_num = target_row.pat_acct_num
                index_filter = source_df[(source_df['Unit_Num'] == unit_num) & (source_df['Patient_Acct_Num'] == patient_acct_num)].index.values
                for index in index_filter:
                    if target_row.iplan_id_ins1 is None:
                        source_df.loc[index, 'BQ_IPlan'] = ""
                    else:
                        source_df.loc[index, 'BQ_IPlan'] = str(int(target_row.iplan_id_ins1)).zfill(5)
                    if target_row.iplan_id_ins2 is None:
                        source_df.loc[index, 'BQ_IPlan2'] = ""
                    else:
                        source_df.loc[index, 'BQ_IPlan2'] = str(int(target_row.iplan_id_ins2)).zfill(5)
                    source_df.loc[index, 'Patient_DW_ID'] = str(target_row.patient_dw_id)
                    source_df.loc[index, 'IPlan1_Match'] = str(source_df.loc[index, 'BQ_IPlan'] == source_df.loc[index, 'IPlan'])
                    source_df.loc[index, 'IPlan2_Match'] = str(source_df.loc[index, 'BQ_IPlan2'] == source_df.loc[index, 'IPlan2'])

        table_rec_counts = []            
        print(source_df.columns.values)
        table_rec_counts.append((',').join(source_df.columns.values))

        for index, row in source_df.iterrows():
            table_rec_counts.append((',').join([row['PAS_COID'], row['Unit_Num'], str(row['Patient_Acct_Num']), row['IPlan'], row['IPlan2']
            , row['BQ_IPlan'], row['BQ_IPlan2'], row['Patient_DW_ID'], row['IPlan1_Match'], row['IPlan2_Match']
            ]))

        write_file_local(output_path, table_rec_counts)
        dt2 = datetime.now()
        print(dt2-dt1)

    except Exception as e1:
        print(e1)
        print("Unit_Num : {}, Patient_Acct_Num : {}".format(unit_num, patient_acct_num))
        pass

In [130]:
print("Begin of Processing")

lookup_bq_data()

print("End of Processing")

Begin of Processing
['PAS_COID' 'Unit_Num' 'Patient_Acct_Num' 'IPlan' 'IPlan2' 'BQ_IPlan'
 'BQ_IPlan2' 'Patient_DW_ID' 'IPlan1_Match' 'IPlan2_Match']
0:08:45.194195
End of Processing
